##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to the Keras Tuner

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/keras_tuner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called *hyperparameter tuning* or *hypertuning*. 

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:
1. **Model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

## Setup

In [2]:
import tensorflow as tf
import os
import gzip
import numpy as np
from tensorflow import keras

import IPython

Install and import the Keras Tuner.

In [3]:
!pip install -q -U keras-tuner
import kerastuner as kt

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


## Download and prepare the dataset

In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist). 

Load the data.

In [4]:
# 定义加载数据的函数，data_folder为保存gz数据的文件夹，该文件夹下有4个文件
# 'train-labels-idx1-ubyte.gz', 'train-images-idx3-ubyte.gz',
# 't10k-labels-idx1-ubyte.gz', 't10k-images-idx3-ubyte.gz'

def load_data(data_folder):

  files = [
      'train-labels-idx1-ubyte.gz', 'train-images-idx3-ubyte.gz',
      't10k-labels-idx1-ubyte.gz', 't10k-images-idx3-ubyte.gz'
  ]

  paths = []
  for fname in files:
    paths.append(os.path.join(data_folder,fname))
    print(paths[0])

  with gzip.open(paths[0], 'rb') as lbpath:
    y_train = np.frombuffer(lbpath.read(), np.uint8, offset=8)

  with gzip.open(paths[1], 'rb') as imgpath:
    x_train = np.frombuffer(
        imgpath.read(), np.uint8, offset=16).reshape(len(y_train), 28, 28)

  with gzip.open(paths[2], 'rb') as lbpath:
    y_test = np.frombuffer(lbpath.read(), np.uint8, offset=8)

  with gzip.open(paths[3], 'rb') as imgpath:
    x_test = np.frombuffer(
        imgpath.read(), np.uint8, offset=16).reshape(len(y_test), 28, 28)

  return (x_train, y_train), (x_test, y_test)

(img_train, label_train), (img_test, label_test) = load_data('./datasets/fashion/')

./datasets/fashion/train-labels-idx1-ubyte.gz
./datasets/fashion/train-labels-idx1-ubyte.gz
./datasets/fashion/train-labels-idx1-ubyte.gz
./datasets/fashion/train-labels-idx1-ubyte.gz


In [5]:
#(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [6]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a *hypermodel*.

You can define a hypermodel through two approaches:

* By using a model builder function
* By subclassing the `HyperModel` class of the Keras Tuner API

You can also use two pre-defined `HyperModel` classes - [HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperxception-class) and [HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class) for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. In this tutorial, you use the [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner. 

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.

Before running the hyperparameter search, define a callback to clear the training outputs at the end of every training step.

In [9]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [10]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.001.



To finish this tutorial, retrain the model with the optimal hyperparameters from the search.

In [11]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10


53984/60000 [=========================>....] - ETA: 13:36 - loss: 2.3805 - accuracy: 0.062 - ETA: 1:16 - loss: 1.7178 - accuracy: 0.395 - ETA: 45s - loss: 1.3846 - accuracy: 0.5028 - ETA: 34s - loss: 1.2166 - accuracy: 0.569 - ETA: 27s - loss: 1.1006 - accuracy: 0.606 - ETA: 24s - loss: 1.0394 - accuracy: 0.634 - ETA: 21s - loss: 0.9688 - accuracy: 0.662 - ETA: 19s - loss: 0.9261 - accuracy: 0.673 - ETA: 18s - loss: 0.8938 - accuracy: 0.684 - ETA: 17s - loss: 0.8649 - accuracy: 0.694 - ETA: 16s - loss: 0.8448 - accuracy: 0.700 - ETA: 15s - loss: 0.8331 - accuracy: 0.705 - ETA: 15s - loss: 0.8134 - accuracy: 0.713 - ETA: 14s - loss: 0.8067 - accuracy: 0.718 - ETA: 14s - loss: 0.7951 - accuracy: 0.720 - ETA: 14s - loss: 0.7823 - accuracy: 0.725 - ETA: 13s - loss: 0.7711 - accuracy: 0.727 - ETA: 13s - loss: 0.7652 - accuracy: 0.729 - ETA: 13s - loss: 0.7575 - accuracy: 0.732 - ETA: 13s - loss: 0.7490 - accuracy: 0.735 - ETA: 12s - loss: 0.7436 - accuracy: 0.738 - ETA: 12s - loss: 0.7363 -

60000/60000 [==============================] - ETA: 1s - loss: 0.4809 - accuracy: 0.82 - ETA: 1s - loss: 0.4805 - accuracy: 0.82 - ETA: 0s - loss: 0.4796 - accuracy: 0.82 - ETA: 0s - loss: 0.4792 - accuracy: 0.82 - ETA: 0s - loss: 0.4786 - accuracy: 0.82 - ETA: 0s - loss: 0.4781 - accuracy: 0.82 - ETA: 0s - loss: 0.4777 - accuracy: 0.82 - ETA: 0s - loss: 0.4772 - accuracy: 0.82 - ETA: 0s - loss: 0.4765 - accuracy: 0.82 - ETA: 0s - loss: 0.4756 - accuracy: 0.83 - ETA: 0s - loss: 0.4758 - accuracy: 0.82 - ETA: 0s - loss: 0.4753 - accuracy: 0.82 - ETA: 0s - loss: 0.4749 - accuracy: 0.83 - ETA: 0s - loss: 0.4744 - accuracy: 0.83 - ETA: 0s - loss: 0.4742 - accuracy: 0.83 - ETA: 0s - loss: 0.4736 - accuracy: 0.83 - ETA: 0s - loss: 0.4731 - accuracy: 0.83 - ETA: 0s - loss: 0.4729 - accuracy: 0.83 - ETA: 0s - loss: 0.4724 - accuracy: 0.83 - 13s 215us/sample - loss: 0.4723 - accuracy: 0.8309 - val_loss: 0.4202 - val_accuracy: 0.8472
Epoch 2/10


60000/60000 [==============================] - ETA: 13s - loss: 0.2998 - accuracy: 0.875 - ETA: 9s - loss: 0.3789 - accuracy: 0.875 - ETA: 9s - loss: 0.3520 - accuracy: 0.87 - ETA: 9s - loss: 0.3553 - accuracy: 0.87 - ETA: 9s - loss: 0.3583 - accuracy: 0.87 - ETA: 9s - loss: 0.3646 - accuracy: 0.86 - ETA: 9s - loss: 0.3547 - accuracy: 0.87 - ETA: 9s - loss: 0.3534 - accuracy: 0.87 - ETA: 9s - loss: 0.3531 - accuracy: 0.87 - ETA: 9s - loss: 0.3631 - accuracy: 0.86 - ETA: 8s - loss: 0.3641 - accuracy: 0.86 - ETA: 8s - loss: 0.3633 - accuracy: 0.86 - ETA: 8s - loss: 0.3611 - accuracy: 0.86 - ETA: 8s - loss: 0.3571 - accuracy: 0.86 - ETA: 8s - loss: 0.3591 - accuracy: 0.86 - ETA: 8s - loss: 0.3602 - accuracy: 0.86 - ETA: 8s - loss: 0.3650 - accuracy: 0.86 - ETA: 8s - loss: 0.3641 - accuracy: 0.86 - ETA: 8s - loss: 0.3629 - accuracy: 0.86 - ETA: 8s - loss: 0.3627 - accuracy: 0.86 - ETA: 8s - loss: 0.3638 - accuracy: 0.86 - ETA: 8s - loss: 0.3711 - accuracy: 0.85 - ETA: 8s - loss: 0.3713 - a

57920/60000 [===========================>..] - ETA: 13s - loss: 0.1445 - accuracy: 0.937 - ETA: 10s - loss: 0.3354 - accuracy: 0.884 - ETA: 10s - loss: 0.2984 - accuracy: 0.898 - ETA: 10s - loss: 0.2892 - accuracy: 0.899 - ETA: 9s - loss: 0.3154 - accuracy: 0.887 - ETA: 9s - loss: 0.3098 - accuracy: 0.88 - ETA: 9s - loss: 0.3112 - accuracy: 0.88 - ETA: 9s - loss: 0.3134 - accuracy: 0.88 - ETA: 9s - loss: 0.3218 - accuracy: 0.88 - ETA: 9s - loss: 0.3299 - accuracy: 0.88 - ETA: 10s - loss: 0.3383 - accuracy: 0.876 - ETA: 10s - loss: 0.3336 - accuracy: 0.879 - ETA: 10s - loss: 0.3320 - accuracy: 0.878 - ETA: 10s - loss: 0.3361 - accuracy: 0.877 - ETA: 10s - loss: 0.3390 - accuracy: 0.877 - ETA: 10s - loss: 0.3330 - accuracy: 0.879 - ETA: 10s - loss: 0.3341 - accuracy: 0.878 - ETA: 10s - loss: 0.3327 - accuracy: 0.878 - ETA: 10s - loss: 0.3313 - accuracy: 0.879 - ETA: 10s - loss: 0.3361 - accuracy: 0.875 - ETA: 10s - loss: 0.3343 - accuracy: 0.875 - ETA: 10s - loss: 0.3362 - accuracy: 0.87

...

57856/60000 [===========================>..] - ETA: 29s - loss: 0.3620 - accuracy: 0.875 - ETA: 16s - loss: 0.2564 - accuracy: 0.886 - ETA: 18s - loss: 0.2927 - accuracy: 0.889 - ETA: 16s - loss: 0.2781 - accuracy: 0.898 - ETA: 14s - loss: 0.2677 - accuracy: 0.900 - ETA: 14s - loss: 0.2833 - accuracy: 0.897 - ETA: 14s - loss: 0.2765 - accuracy: 0.894 - ETA: 14s - loss: 0.2707 - accuracy: 0.895 - ETA: 13s - loss: 0.2682 - accuracy: 0.901 - ETA: 12s - loss: 0.2642 - accuracy: 0.903 - ETA: 11s - loss: 0.2621 - accuracy: 0.903 - ETA: 11s - loss: 0.2571 - accuracy: 0.903 - ETA: 11s - loss: 0.2506 - accuracy: 0.906 - ETA: 11s - loss: 0.2508 - accuracy: 0.905 - ETA: 11s - loss: 0.2496 - accuracy: 0.905 - ETA: 11s - loss: 0.2463 - accuracy: 0.907 - ETA: 11s - loss: 0.2438 - accuracy: 0.909 - ETA: 11s - loss: 0.2443 - accuracy: 0.909 - ETA: 11s - loss: 0.2460 - accuracy: 0.909 - ETA: 11s - loss: 0.2450 - accuracy: 0.909 - ETA: 11s - loss: 0.2459 - accuracy: 0.909 - ETA: 11s - loss: 0.2442 - acc

60000/60000 [==============================] - ETA: 0s - loss: 0.2538 - accuracy: 0.90 - ETA: 0s - loss: 0.2538 - accuracy: 0.90 - ETA: 0s - loss: 0.2536 - accuracy: 0.90 - ETA: 0s - loss: 0.2535 - accuracy: 0.90 - ETA: 0s - loss: 0.2531 - accuracy: 0.90 - ETA: 0s - loss: 0.2529 - accuracy: 0.90 - 12s 195us/sample - loss: 0.2528 - accuracy: 0.9061 - val_loss: 0.3347 - val_accuracy: 0.8834
Epoch 8/10
60000/60000 [==============================] - ETA: 16s - loss: 0.2404 - accuracy: 0.968 - ETA: 10s - loss: 0.2642 - accuracy: 0.900 - ETA: 10s - loss: 0.2378 - accuracy: 0.917 - ETA: 10s - loss: 0.2514 - accuracy: 0.911 - ETA: 10s - loss: 0.2558 - accuracy: 0.905 - ETA: 10s - loss: 0.2768 - accuracy: 0.899 - ETA: 10s - loss: 0.2791 - accuracy: 0.902 - ETA: 10s - loss: 0.2727 - accuracy: 0.904 - ETA: 10s - loss: 0.2645 - accuracy: 0.906 - ETA: 10s - loss: 0.2612 - accuracy: 0.904 - ETA: 9s - loss: 0.2573 - accuracy: 0.905 - ETA: 9s - loss: 0.2569 - accuracy: 0.90 - ETA: 9s - loss: 0.2529 - 

60000/60000 [==============================] - ETA: 13s - loss: 0.1952 - accuracy: 0.875 - ETA: 9s - loss: 0.2315 - accuracy: 0.906 - ETA: 9s - loss: 0.2454 - accuracy: 0.90 - ETA: 9s - loss: 0.2338 - accuracy: 0.91 - ETA: 8s - loss: 0.2403 - accuracy: 0.90 - ETA: 8s - loss: 0.2372 - accuracy: 0.90 - ETA: 8s - loss: 0.2395 - accuracy: 0.90 - ETA: 8s - loss: 0.2393 - accuracy: 0.90 - ETA: 8s - loss: 0.2374 - accuracy: 0.90 - ETA: 8s - loss: 0.2383 - accuracy: 0.91 - ETA: 8s - loss: 0.2422 - accuracy: 0.90 - ETA: 8s - loss: 0.2450 - accuracy: 0.90 - ETA: 8s - loss: 0.2442 - accuracy: 0.90 - ETA: 8s - loss: 0.2429 - accuracy: 0.90 - ETA: 8s - loss: 0.2423 - accuracy: 0.90 - ETA: 8s - loss: 0.2407 - accuracy: 0.90 - ETA: 8s - loss: 0.2379 - accuracy: 0.91 - ETA: 8s - loss: 0.2380 - accuracy: 0.91 - ETA: 8s - loss: 0.2382 - accuracy: 0.91 - ETA: 8s - loss: 0.2356 - accuracy: 0.91 - ETA: 8s - loss: 0.2337 - accuracy: 0.91 - ETA: 8s - loss: 0.2331 - accuracy: 0.91 - ETA: 8s - loss: 0.2300 - a

54592/60000 [==========================>...] - ETA: 14s - loss: 0.1411 - accuracy: 0.968 - ETA: 9s - loss: 0.2554 - accuracy: 0.906 - ETA: 8s - loss: 0.2569 - accuracy: 0.90 - ETA: 8s - loss: 0.2416 - accuracy: 0.90 - ETA: 8s - loss: 0.2240 - accuracy: 0.91 - ETA: 8s - loss: 0.2295 - accuracy: 0.91 - ETA: 8s - loss: 0.2257 - accuracy: 0.91 - ETA: 8s - loss: 0.2148 - accuracy: 0.91 - ETA: 8s - loss: 0.2214 - accuracy: 0.91 - ETA: 8s - loss: 0.2259 - accuracy: 0.91 - ETA: 8s - loss: 0.2264 - accuracy: 0.91 - ETA: 8s - loss: 0.2254 - accuracy: 0.91 - ETA: 8s - loss: 0.2270 - accuracy: 0.91 - ETA: 8s - loss: 0.2224 - accuracy: 0.91 - ETA: 8s - loss: 0.2217 - accuracy: 0.91 - ETA: 8s - loss: 0.2248 - accuracy: 0.91 - ETA: 8s - loss: 0.2258 - accuracy: 0.91 - ETA: 8s - loss: 0.2267 - accuracy: 0.91 - ETA: 8s - loss: 0.2273 - accuracy: 0.91 - ETA: 8s - loss: 0.2278 - accuracy: 0.91 - ETA: 8s - loss: 0.2252 - accuracy: 0.91 - ETA: 8s - loss: 0.2245 - accuracy: 0.91 - ETA: 8s - loss: 0.2236 - a

60000/60000 [==============================] - ETA: 0s - loss: 0.2206 - accuracy: 0.91 - ETA: 0s - loss: 0.2205 - accuracy: 0.91 - ETA: 0s - loss: 0.2203 - accuracy: 0.91 - ETA: 0s - loss: 0.2207 - accuracy: 0.91 - ETA: 0s - loss: 0.2206 - accuracy: 0.91 - ETA: 0s - loss: 0.2206 - accuracy: 0.91 - ETA: 0s - loss: 0.2207 - accuracy: 0.91 - ETA: 0s - loss: 0.2207 - accuracy: 0.91 - ETA: 0s - loss: 0.2209 - accuracy: 0.91 - ETA: 0s - loss: 0.2210 - accuracy: 0.91 - ETA: 0s - loss: 0.2213 - accuracy: 0.91 - ETA: 0s - loss: 0.2212 - accuracy: 0.91 - ETA: 0s - loss: 0.2211 - accuracy: 0.91 - ETA: 0s - loss: 0.2214 - accuracy: 0.91 - ETA: 0s - loss: 0.2216 - accuracy: 0.91 - ETA: 0s - loss: 0.2218 - accuracy: 0.91 - ETA: 0s - loss: 0.2217 - accuracy: 0.91 - ETA: 0s - loss: 0.2218 - accuracy: 0.91 - ETA: 0s - loss: 0.2220 - accuracy: 0.91 - ETA: 0s - loss: 0.2219 - accuracy: 0.91 - ETA: 0s - loss: 0.2217 - accuracy: 0.91 - ETA: 0s - loss: 0.2216 - accuracy: 0.91 - ETA: 0s - loss: 0.2217 - accu

The `my_dir/intro_to_kt` directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite = True` argument while instantiating the tuner.

## Summary

In this tutorial, you learned how to use the Keras Tuner to tune hyperparameters for a model. To learn more about the Keras Tuner, check out these additional resources:

* [Keras Tuner on the TensorFlow blog](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* [Keras Tuner website](https://keras-team.github.io/keras-tuner/)

Also check out the [HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) in TensorBoard to interactively tune your model hyperparameters.